# Réarrengement data Solordinario

In [1]:
import numpy as np
import glob 
import os.path
import scipy.io.wavfile as wavfile

In [2]:
path = 'data'

def listdirectory(path): 
    fichier=[] 
    l = glob.glob(path+'\\*') 
    for i in l: 
        if os.path.isdir(i): fichier.extend(listdirectory(i)) 
        else: fichier.append(i) 
    return fichier

files = listdirectory(path)

In [3]:
long = 3 #durée à conserver en sec
for file in files:
    Fe, sig = wavfile.read(file)
    new_dir = 'data_2/'+file[5:]
    wavfile.write(new_dir, Fe, sig[:long*Fe])

# Dictionnaire avec Pitch et Velocite

In [4]:
import descriptors
import torch
import numpy as np

In [6]:
import warnings
warnings.filterwarnings('ignore')

path = 'data_2'
files = listdirectory(path)
nb_files = len(files)
print(nb_files)
sample_per_sec = 250

dict_env = {'file_name' : [[]]*nb_files, 'env' : [[]]*nb_files, 'vel' : [[]]*nb_files, 'pitch' : [[]]*nb_files, 'corde' : [[]]*nb_files}
f = 0

std_tab = np.zeros(len(files))

for file in files:
    name = file[7:]
    vel = file[14:16]
    corde = file[17:19]
    if file[-6]=='#':
        pitch = file[-7:-4]
    else :
        pitch = file[-6:-4]
        
    Fe, sig = wavfile.read(file)
    sig = sig.astype(np.float32)
    env = np.zeros([2,long*sample_per_sec])
    env[0,:] = descriptors.extract_loudness(sig, sampling_rate=Fe, block_size=int(Fe/250), n_fft=2048)[:long*sample_per_sec]
    env[1,:] = descriptors.extract_pitch(sig, sampling_rate=Fe, block_size=int(Fe/250))[:long*sample_per_sec]
    print(np.std(env[1,:]))
    std_tab[f] = np.std(env[1,:])
    
    dict_env['file_name'][f] = file
    dict_env['env'][f] = env
    dict_env['vel'][f] = vel
    dict_env['pitch'][f] = pitch
    dict_env['corde'][f] = corde
    print(f)
    f += 1
  

279
396.8684706222648
0
8.052408242947921
1
229.58827541051699
2
2.000034974680111
3
178.70506619621105
4
4.742532806241556
5
3.010884987861959
6
144.64996872979142
7
0.6084882852704265
8
6.583868572026283
9
4.857492511194998
10
2.46799005987597
11
4.284235705537222
12
2.052137004997727
13
8.382031261529617
14
3.0289302133219502
15
5.197349139866898
16
0.965942832305004
17
5.302804303092014
18
4.666417501560996
19
1.314002932799615
20
8.683188680236817
21
3.167903825676519
22
0.9470540154303309
23
4.441590587489021
24
1.0697462203306733
25
2.5559608928378625
26
2.484600499188226
27
1.8724245435737794
28
29.914407973784154
29
9.919318820524554
30
1.2622216516521823
31
2.7029814172516247
32
41.491728834964796
33
4.4889270989836065
34
30.817070944179633
35
5.158063414486158
36
1.2246007676721098
37
3.8564830204701708
38
1.7789504837714902
39
4.44948361947553
40
50.36273427411788
41
8.157512318969479
42
1.8228450293341363
43
7.458544808901986
44
2.4441361997304285
45
6.568367344366547
46
1

In [39]:
dict_r = np.load('data_dict.npy', allow_pickle=True).item()

dict_r['env'][200]

array([[9.91890564e-01, 1.26108655e+00, 1.39861906e+00, ...,
        3.83027832e+00, 3.96299890e+00, 4.12451463e+00],
       [2.77347106e+02, 2.79756641e+02, 2.79565720e+02, ...,
        1.33018590e+03, 1.33051019e+03, 1.33211754e+03]])

In [40]:
f=0
for std in std_tab:
    print(f)
    if std > 50:
        del dict_r['env'][f]
        del dict_r['file_name'][f]
        del dict_r['vel'][f]
        del dict_r['pitch'][f]
        del dict_r['corde'][f]
        f-=1
    f += 1
        

0
0
1
1
2
2
3
4
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
91
92
92
92
92
93
94
94
95
96
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
179
180
180
181
181
182
182
183
184
184
185
186
187
187
188
189
190
191
192
193
194
195
196
197
198
199
199
200
201
202
202
203
204
205
205
206
206
207
207
208
209
209
210
211
212
213
214
215
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
246
247
248
249
250
251


In [42]:
# Save
np.save('data_dict.npy', dict_r) 

# Load
# dict_r = np.load('data_dict.npy').item() 

# Création dataset pour torch (pas encore fait, les codes ici sont des copiés/collés, mais je n'ai pas encore travaillé dessus)